# Exploring_CENSO_dataset

## Content
- [VIVIENDA](#VIVIENDA)
- [HOGAR](#HOGAR)
- [PERSONAS](#PERSONAS)
- [FALLECIDOS](#FALLECIDOS)
- [GEOREFERENCIACION](#GEOREFERENCIACION)
- [Divipola](#Divipola)

## Goal of visualization

## Summary about  each dataset


In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os

In [2]:
os.chdir(os.path.join(os.getcwd(), "..", "src"))

## Inputs

In [3]:
department = "Antioquia"

## Paths

In [4]:
data_path = os.path.join(os.getcwd(), "..", "data")
censo_data = os.path.join(data_path, "censo")

dict_paths_departments = {}
for name in os.listdir(censo_data):
    if os.path.isdir(os.path.join(censo_data, name)):
        dict_paths_departments[name[3:]] = os.path.join(censo_data, name, name + "_CSV")

## Samples

In [5]:
current_dict_files = {}
for file in os.listdir(dict_paths_departments[department]):
    if "VIV" in file:
        current_dict_files["VIV"] = os.path.join(dict_paths_departments[department],file)
    elif "HOG" in file:
        current_dict_files["HOG"] = os.path.join(dict_paths_departments[department],file)
    elif "PER" in file:
        current_dict_files["PER"] = os.path.join(dict_paths_departments[department],file)
    elif "FALL" in file:
        current_dict_files["FALL"] = os.path.join(dict_paths_departments[department],file)
    elif "MGN" in file:
        current_dict_files["MGN"] = os.path.join(dict_paths_departments[department],file)

In [6]:
current_dict_files

{'VIV': '/Users/camilovelasquez/Desktop/Documents/Camilo/Udacity/censo_covid_colombia/src/../data/censo/05_Antioquia/05_Antioquia_CSV/CNPV2018_1VIV_A2_05.CSV',
 'HOG': '/Users/camilovelasquez/Desktop/Documents/Camilo/Udacity/censo_covid_colombia/src/../data/censo/05_Antioquia/05_Antioquia_CSV/CNPV2018_2HOG_A2_05.CSV',
 'PER': '/Users/camilovelasquez/Desktop/Documents/Camilo/Udacity/censo_covid_colombia/src/../data/censo/05_Antioquia/05_Antioquia_CSV/CNPV2018_5PER_A2_05.CSV',
 'FALL': '/Users/camilovelasquez/Desktop/Documents/Camilo/Udacity/censo_covid_colombia/src/../data/censo/05_Antioquia/05_Antioquia_CSV/CNPV2018_3FALL_A2_05.CSV',
 'MGN': '/Users/camilovelasquez/Desktop/Documents/Camilo/Udacity/censo_covid_colombia/src/../data/censo/05_Antioquia/05_Antioquia_CSV/CNPV2018_MGN_A2_05.CSV'}

----

### VIVIENDA
[Volver](#Exploring_CENSO_dataset)

**Columns:**
- **'TIPO_REG':** Tipo de registro
- **'U_DPTO':** Departamento
- **'U_MPIO':** Municipio
- **'UA_CLASE':** Clase
    - 1_Cabecera Municipal
    - 2_Centro Poblado
    - 3_Rural Disperso
    - 4_Resto Rural (2 y 3 )
- **'U_EDIFICA':** Número de orden de la edificación
- **'COD_ENCUESTAS':** Código encuesta
- **'U_VIVIENDA':** Numero de orden de la Vivienda
- **'UVA_ESTATER':** Vivienda en una territorialidad étnica
- **'UVA1_TIPOTER':** Tipo de territorialidad étnica
- **'UVA2_CODTER':** Código de territorialidad étnica
- **'UVA_ESTA_AREAPROT':** Vivienda en un área protegida
- **'UVA1_COD_AREAPROT':** Codigo area protegida
- **'UVA_USO_UNIDAD':** Uso de la unidad
    - 1 Vivienda
    - 2 Mixto (Espacio independiente y separado que combina vivienda con otro uso no residencial)
    - 3 Unidad NO Residencial (Espacio independiente y separado con uso <> vivienda)
    - 4 Lugar Especial de Alojamiento - LEA
- **'V_TIPO_VIV':** Tipo de vivienda
    - 1 Casa
    - 2 Apartamento
    - 3 Tipo cuarto
    - 4 Vivienda tradicional Indigena
    - 5 Vivienda tradicional Etnica (Afrocolombiana, Isleña, Rrom)
    - 6 Otro (contenedor, carpa, embarcacion, vagon, cueva, refugio natural, etc.)
- **'V_CON_OCUP':** Condicion de ocupacion
    - 1 Ocupada con personas presentes
    - 2 Ocupada con todas las personas ausentes
    - 3 Vivienda temporal (para vacaciones, trabajo etc.)
    - 4 Desocupada
- **'V_TOT_HOG':** Total de hogares en la vivienda
    - 0.Nro_Hogar 00
    - 1.Nro_Hogar 01
    - 2.Nro_Hogar 02
    - ...
    - 25.Nro_Hogar 25
- **'V_MAT_PARED':** Material predominante en paredes exteriores
    - 1 Bloque, ladrillo, piedra, madera pulida
    - 2 Concreto vaciado
    - 3 Material prefabricado
    - 4 Guadua
    - 5 Tapia pisada, bahareque, adobe
    - 6 Madera burda, tabla, tablon
    - 7 Caña, esterilla, otros vegetales
    - 8 Materiales de deshecho (Zinc, tela, carton, latas, plasticos, otros)
    - 9 No tiene paredes
- **'V_MAT_PISO':** Material predominante en los pisos
    - 1 Marmol, parque, madera pulida y lacada
    - 2 Baldosa, vinilo, tableta, ladrillo, laminado
    - 3 Alfombra
    - 4 Cemento, gravilla
    - 5 Madera burda, tabla, tablon, otro vegetal
    - 6 Tierra, arena, barro
- **'VA_EE':** Cuenta con servicio de energía eléctrica
    - 1 Si
    - 2 No
- **'VA1_ESTRATO':** Estrato de la vivienda (según servicio de energía)
    - 0 Sin Estrato
    - 1 Estrato 1
    - 2 Estrato 2
    - 3 Estrato 3
    - 4 Estrato 4
    - 5 Estrato 5
    - 6 Estrato 6
    - 9 No sabe el estrato
- **'VB_ACU':** Cuenta con servicio de acueducto
    - 1 Si
    - 2 No
- **'VC_ALC':** Cuenta con servicio de alcantarillado
    - 1 Si
    - 2 No
- **'VD_GAS':** Cuenta con servicio de gas natural conectado a red pública
    - 1 Si
    - 2 No
    - 9 No informa
- **'VE_RECBAS':** Cuenta con servicio de recolección de basura
    - 1 Si
    - 2 No
    - 9 No informa
- **'VE1_QSEM':** Cuántas veces por semana (recolección basura)
    - 1 1 Vez
    - 2 2 Veces
    - 3 3 Veces
    - 4 4 Veces
    - 5 5 Veces
    - 6 6 Veces
    - 7 7 Veces
    - 8 Mayor periodicidad
    - 9 No Sabe
    - No aplica
- **'VF_INTERNET':** Cuenta con servicio de internet (fijo o móvil)
    - 1 Si
    - 2 No
- **'V_TIPO_SERSA':** Tipo de servicio sanitario (inodoro)
    - 1 Inodoro conectado al alcantarillado?
    - 2 Inodoro conectado a pozo septico?
    - 3 Inodoro sin conexion?
    - 4 Letrina?
    - 5 Inodoro con descarga directa a fuentes de agua (bajamar)?
    - 6 esta vivienda No tiene servicio sanitario?
- **'L_TIPO_INST':** LEA_Tipo de institución o establecimiento
    - 1 Centro penitenciario
    - 2 Institucion de proteccion e internado preventivo para niños, niñas y adolescentes
    - 3 Centro de proteccion y atencion al adulto mayor
    - 4 Convento, seminario, monasterio u otras instituciones similiares
    - 5 Sede educativa con Poblacion interna
    - 6 Cuartel, guarnicion militar (Ejercito, Armada y Fuerza AErea)
    - 7 Comando de policIa, estacion de policIa
    - 8 Campamento de trabajo
    - 9 Casa de lenocinio o prostIbulo
    - 10 Albergue de desplazados
    - 11 Hogar de paz
    - 12 Centro de rehabilitacion funcional
    - 13 Casa de paso indigena
    - No Aplica
- **'L_EXISTEHOG':** LEA_Existencia de hogar
    - 1 Si
    - 2 No
    - 9 No Informa
    - No Aplica
- **'L_TOT_PERL':** LEA_Total de residentes (no pertenecen a hogares)

In [7]:
df_viv = pd.read_csv(current_dict_files["VIV"])
df_viv.head()

,TIPO_REG,U_DPTO,U_MPIO,UA_CLASE,U_EDIFICA,COD_ENCUESTAS,U_VIVIENDA,UVA_ESTATER,UVA1_TIPOTER,UVA2_CODTER,...,VB_ACU,VC_ALC,VD_GAS,VE_RECBAS,VE1_QSEM,VF_INTERNET,V_TIPO_SERSA,L_TIPO_INST,L_EXISTEHOG,L_TOT_PERL
0,1,5,1,1,1,3646,5,2,NaN,NaN,...,1.0,1.0,1.0,1.0,2.0,2.0,1.0,NaN,NaN,NaN
1,1,5,1,1,1,3649,6,2,NaN,NaN,...,1.0,1.0,1.0,1.0,2.0,1.0,1.0,NaN,NaN,NaN
2,1,5,1,1,1,3658,38,2,NaN,NaN,...,1.0,1.0,2.0,1.0,2.0,1.0,1.0,NaN,NaN,NaN
3,1,5,1,1,1,3662,40,2,NaN,NaN,...,1.0,1.0,1.0,1.0,2.0,1.0,1.0,NaN,NaN,NaN
4,1,5,1,1,1,3666,39,2,NaN,NaN,...,1.0,1.0,2.0,1.0,2.0,1.0,1.0,NaN,NaN,NaN


In [8]:
df_viv.shape

(2324080, 30)

In [9]:
df_viv.groupby(["COD_ENCUESTAS", "U_VIVIENDA"]).count()

,,TIPO_REG,U_DPTO,U_MPIO,UA_CLASE,U_EDIFICA,UVA_ESTATER,UVA1_TIPOTER,UVA2_CODTER,UVA_ESTA_AREAPROT,UVA1_COD_AREAPROT,...,VB_ACU,VC_ALC,VD_GAS,VE_RECBAS,VE1_QSEM,VF_INTERNET,V_TIPO_SERSA,L_TIPO_INST,L_EXISTEHOG,L_TOT_PERL
COD_ENCUESTAS,U_VIVIENDA,,,,,,,,,,,,,,,,,,,,,
1,1,1,1,1,1,1,1,0,0,1,0,...,1,1,1,1,1,1,1,0,0,0
2,2,1,1,1,1,1,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,2,1,1,1,1,1,1,0,0,1,0,...,1,1,1,1,1,1,1,0,0,0
5,1,1,1,1,1,1,1,0,0,1,0,...,1,1,1,1,1,1,1,0,0,0
11,1,1,1,1,1,1,1,0,0,1,0,...,1,1,1,1,1,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
950003191,4,1,1,1,1,1,1,0,0,1,0,...,0,0,0,0,0,0,0,1,1,1
950003212,1,1,1,1,1,1,1,0,0,1,0,...,0,0,0,0,0,0,0,1,1,1
950003220,1,1,1,1,1,1,1,0,0,1,0,...,0,0,0,0,0,0,0,1,1,1


In [10]:
df_viv.dtypes

TIPO_REG               int64
U_DPTO                 int64
U_MPIO                 int64
UA_CLASE               int64
U_EDIFICA              int64
COD_ENCUESTAS          int64
U_VIVIENDA             int64
UVA_ESTATER            int64
UVA1_TIPOTER         float64
UVA2_CODTER          float64
UVA_ESTA_AREAPROT      int64
UVA1_COD_AREAPROT    float64
UVA_USO_UNIDAD         int64
V_TIPO_VIV           float64
V_CON_OCUP           float64
V_TOT_HOG            float64
V_MAT_PARED          float64
V_MAT_PISO           float64
VA_EE                float64
VA1_ESTRATO          float64
VB_ACU               float64
VC_ALC               float64
VD_GAS               float64
VE_RECBAS            float64
VE1_QSEM             float64
VF_INTERNET          float64
V_TIPO_SERSA         float64
L_TIPO_INST          float64
L_EXISTEHOG          float64
L_TOT_PERL           float64
dtype: object

In [11]:
df_viv.isnull().sum()

TIPO_REG                   0
U_DPTO                     0
U_MPIO                     0
UA_CLASE                   0
U_EDIFICA                  0
COD_ENCUESTAS              0
U_VIVIENDA                 0
UVA_ESTATER                0
UVA1_TIPOTER         2316994
UVA2_CODTER          2316994
UVA_ESTA_AREAPROT          0
UVA1_COD_AREAPROT    2302918
UVA_USO_UNIDAD             0
V_TIPO_VIV              1794
V_CON_OCUP              1794
V_TOT_HOG             390497
V_MAT_PARED           390497
V_MAT_PISO            390497
VA_EE                 390497
VA1_ESTRATO           413694
VB_ACU                390497
VC_ALC                390497
VD_GAS                390497
VE_RECBAS             390497
VE1_QSEM              603315
VF_INTERNET           390497
V_TIPO_SERSA          390497
L_TIPO_INST          2322286
L_EXISTEHOG          2322286
L_TOT_PERL           2322286
dtype: int64

In [12]:
df_viv.shape

(2324080, 30)

----

### HOGAR
[Volver](#Exploring_CENSO_dataset)

**Columns:**
- **'TIPO_REG':** Tipo de registro
- **'U_DPTO':** Departamento
- **'U_MPIO':** Municipio
- **'UA_CLASE':** Clase
    - 1_Cabecera Municipal
    - 2_Centro Poblado
    - 3_Rural Disperso
    - 4_Resto Rural (2 y 3 )
- **'COD_ENCUESTAS':** Codigo Encuesta
- **'U_VIVIENDA':** Numero roden de la vivienda
- **'H_NROHOG':** Número de hogar en la vivienda
    - 0.Nro_Hogar 00
    - 1.Nro_Hogar 01
    - 2.Nro_Hogar 02
    - ...
    - 25.Nro_Hogar 25
- **'H_NRO_CUARTOS':** Número de cuartos en total
    - 1 1 cuartos
    - 2 2 cuartos
    - 3 3 cuartos
    - 4 4 cuartos
    - ...
    - 20 20 cuartos
    - 99 No informa
- **'H_NRO_DORMIT':** Número de cuartos para dormir
    - 1 1 cuartos
    - 2 2 cuartos
    - 3 3 cuartos
    - ...
    - 20 20 cuartos
    - 99 No informa
- **'H_DONDE_PREPALIM':** Lugar donde preparan los alimentos
    - 1 En un cuarto usado solo para cocinar?
    - 2 En un cuarto usado también para dormir?
    - 3 En una sala-comedor con lavaplatos?
    - 4 En una sala-comedor sin lavaplatos?
    - 5 En un patio, corredor, enramada o al aire libre?
    - 6 No preparan alimentos en la vivienda?
    - 9 No informa
- **'H_AGUA_COCIN':** Fuente de agua para preparar los alimentos
    - 1 Acueducto público?
    - 2 Acueducto veredal?
    - 3 Red de distribución comunitaria?
    - 4 Pozo con bomba?
    - 5 Pozo sin bomba, aljibe, jaguey o barreno?
    - 6 Agua lluvia?
    - 7 RÍo, quebrada, manantial, nacimiento?
    - 8 Pila pública?
    - 9 Carrotanque?
    - 10 Aguatero?
    - 11 Agua embotellada o en bolsa?
    - 12 no preparan alimentos
    - 99 No informa
- **'HA_NRO_FALL':** Total fallecidos en el hogar (2017)
    - 0 00 Personas
    - 1 01 Persona
    - 2 02 Personas
    - ...
    - 20 20 Personas
- **'HA_TOT_PER':** Total personas en el hogar
    - 1 01 Persona
    - 2 02 Personas
    - 3 03 Personas
    - ...
    - 40 40 Personas

In [13]:
df_hog = pd.read_csv(current_dict_files["HOG"])
df_hog.head()

,TIPO_REG,U_DPTO,U_MPIO,UA_CLASE,COD_ENCUESTAS,U_VIVIENDA,H_NROHOG,H_NRO_CUARTOS,H_NRO_DORMIT,H_DONDE_PREPALIM,H_AGUA_COCIN,HA_NRO_FALL,HA_TOT_PER
0,2,5,1,1,3626,4,1.0,4.0,2.0,1.0,1.0,NaN,5.0
1,2,5,1,1,3629,2,1.0,4.0,3.0,1.0,1.0,NaN,3.0
2,2,5,1,1,3630,1,1.0,3.0,2.0,1.0,1.0,NaN,3.0
3,2,5,1,1,3630,1,2.0,3.0,2.0,1.0,1.0,NaN,4.0
4,2,5,1,1,3632,6,1.0,3.0,2.0,1.0,1.0,NaN,3.0


In [15]:
df_hog.isnull().sum()

TIPO_REG                  0
U_DPTO                    0
U_MPIO                    0
UA_CLASE                  0
COD_ENCUESTAS             0
U_VIVIENDA                0
H_NROHOG               1794
H_NRO_CUARTOS          1794
H_NRO_DORMIT           1794
H_DONDE_PREPALIM       1794
H_AGUA_COCIN          17469
HA_NRO_FALL         1955762
HA_TOT_PER             1794
dtype: int64

In [16]:
df_hog.shape

(1985360, 13)

----

### PERSONAS
[Volver](#Exploring_CENSO_dataset)

**Columns:**
- **'TIPO_REG':** Tipo de Registro
- **'U_DPTO':** Departamento
- **'U_MPIO':** Municipio
- **'UA_CLASE':** Clase
    - 1_Cabecera Municipal
    - 2_Centro Poblado
    - 3_Rural Disperso
    - 4_Resto Rural (2 y 3 )
- **'U_EDIFICA':** Numero de orden de la edificación
- **'COD_ENCUESTAS':** Codigo Encuesta
- **'U_VIVIENDA':** Numero de orden de la vivienda
- **'P_NROHOG':** Número de orden del Hogar dentro de la vivienda
    - 0 Nro_Hogar 00
    - 1 Nro_Hogar 01
    - 2 Nro_Hogar 02
    - ...
    - 35 Nro_Hogar 35
- **'P_NRO_PER':** Número de persona en el hogar
    - 1 1
    - 2 2
    - 3 3
    - ...
    - 25 25
- **'P_SEXO':** Sexo
    - 1 Hombre
    - 2 Mujer
- **'P_EDADR':** Edad en Grupos Quinquenales
    - 1 de 00 A 04 Años
    - 2 de 05 A 09 Años
    - 3 de 10 A 14 Años
    - 4 de 15 A 19 Años
    - 5 de 20 A 24 Años
    - 6 de 25 A 29 Años
    - 7 de 30 A 34 Años
    - 8 de 35 A 39 Años
    - 9 de 40 A 44 Años
    - 10 de 45 A 49 Años
    - 11 de 50 A 54 Años
    - 12 de 55 A 59 Años
    - 13 de 60 A 64 Años
    - 14 de 65 A 69 Años
    - 15 de 70 A 74 Años
    - 16 de 75 A 79 Años
    - 17 de 80 A 84 Años
    - 18 de 85 A 89 Años
    - 19 de 90 A 94 Años
    - 20 de 95 A 99 Años
    - 21 de 100 y más Años
- **'P_PARENTESCOR':** Relación de parentesco con el jefe(a) del hogar (recodificada)
    - 1 Jefe(a) del hogar
    - 2 Pareja (Conyuge, compañero(a), esposo(a))
    - 3 Hijo(a), Hijastro(a)
    - 4 Otros Parientes
    - 5 Otros NO parientes
    - No Aplica
- **'PA1_GRP_ETNIC':** Reconocimiento étnico
    - 1 Indígena?
    - 2 Gitano(a) o Rrom?
    - 3 Raizal del Archipielago de San Andrés, Providencia y Santa Catalina?
    - 4 Palenquero(a) de San Basilio?
    - 5 Negro(a), Mulato(a), Afrodescendiente, Afrocolombiano(a)?
    - 6 Ningún grupo étnico
    - 9 No Informa
- **'PA11_COD_ETNIA':** Pueblo indigena pertenciente
- **'PA12_CLAN':** Clan de pertenencia
- **'PA21_COD_VITSA':** Vitsa
- **'PA22_COD_KUMPA':** Kumpania
- **'PA_HABLA_LENG':** Habla lengua Nativa de  su pueblo
- **'PA1_ENTIENDE':** Entiende lengua nativa
- **'PB_OTRAS_LENG':** Habla otra lengua nativa
- **'PB1_QOTRAS_LENG':** Numero de otras lenguas nativas
- **'PA_LUG_NAC':** Lugar de Nacimiento
- **'PA_VIVIA_5ANOS':** Lugar de residencia hace 5 años
    - 1 No había nacido
    - 2 En este mpio
    - 3 En otro mpio colombiano
    - 4 En otro país
    - 9 No Informa
    - No Aplica
- **'PA_VIVIA_1ANO':** Lugar de residencia hace 12 meses
    - 1 No había nacido
    - 2 En este mpio
    - 3 En otro mpio colombiano
    - 4 En otro país
    - 9 No Informa
    - No Aplica
- **'P_ENFERMO':** Algun problema de salud en los últimos 30 días, sin hospitalización
    - 1 SI
    - 2 NO
    - 9 No Informa
- **'P_QUEHIZO_PPAL':** Tratamiento principal del problema de salud
    - 1 Acudió a la entidad de seguridad social en salud de la cual es filiado(a)?
    - 2 Acudió a un médico particular? (general, especialista, odontólogo, terapeuta u otro)
    - 3 Acudió a un boticario, farmacéuta, droguista?
    - 4 Asistió a terapias alternativas? (acupuntura, esencias florales, musicoterapias, homeópata, etc.)
    - 5 Acudió a una autoridad indígena espiritual?
    - 6 Acudió a otro médico de un grupo étnico? (curandero, yerbatero, etc.)
    - 7 Usó remedios caseros?
    - 8 Se autorrecetó
    - 9 No hizo nada
    - 99 No Informa
    - No Aplica
- **'PA_LO_ATENDIERON':** Atención del problema de salud
    - 1 SI
    - 2 NO
    - 9 No Informa
    - No Aplica
- **'PA1_CALIDAD_SERV':** Calidad de la prestación del servicio de salud
    - 1 Muy bueno
    - 2 Bueno
    - 3 Malo
    - 4 Muy Malo
    - 9 No Informa
    - No Aplica
- **'CONDICION_FISICA':** Alguna dificultad en su vida diaria
    - 1 SI
    - 2 NO
    - 9 No Informa
- **'P_ALFABETA':** Sabe leer y escribir
    - 1 SI
    - 2 NO
    - 9 No Informa
    - No Aplic
- **'PA_ASISTENCIA':** Asistencia escolar (de forma presencial o virtual)
    - 1 SI
    - 2 NO
    - 9 No Informa
    - No Aplica
- **'P_NIVEL_ANOSR':** Nivel educativo más alto alcanzado y último año o grado aprobado en ese nivel (recodificado)
    - 1 Preescolar
    - 2 Básica primaria
    - 3 Básica secundaria
    - 4 Media academica o clasica
    - 5 Media tecnica
    - 6 Normalista
    - 7 Técnica profesional o Tecnológica
    - 8 Universitario
    - 9 Especialización, maestría, doctorado
    - 10 Ninguno
    - 99 No Informa
    - No Aplica
- **'P_TRABAJO':** Que hizo durante la semana pasada
    - 1 Trabajó por lo menos una hora en una actividad que le generó algún ingreso?
    - 2 Trabajó o ayudó en un negocio por lo menos una hora sin que le pagaran?
    - 3 No trabajó, pero tenía un empleo, trabajo o negocio por el que recibe ingresos?
    - 4 Busco trabajo?
    - 5 Vivió de jubilación, pensión o renta?
    - 6 Estudió?
    - 7 Realizó oficios del hogar?
    - 8 Está incapacitado(a) permanentemente para trabajar?
    - 9 Estuvo en otra situación?
    - 0 No informa
    - No aplica
- **'P_EST_CIVIL':** Estado civil
    - 1 Unión libre?
    - 2 Casado(a)?
    - 3 Divorciado(a)?
    - 4 Separado(a) de unión libre?
    - 5 Separado(a) de matrimonio?
    - 6 Viudo(a)?
    - 7 Soltero(a)?(Nunca se ha casado, ni ha vivido en unión libre)
    - 9 No Informa
    - No Aplica
- **'PA_HNV':** Ha tenido algún hijo(a) nacido vivo(a)
    - 1 Si
    - 2 No
    - 9 No Informa
    - No Aplica
- **'PA1_THNV':** Hijos(as) nacidos vivos: Total
    - 1 1 Hijo
    - 2 2 Hijos
    - ...
    - 25 25 Hijos
    - No informa 99
    - No aplica
- **'PA2_HNVH':** Hijos(as) nacidos vivos: Hombres
    - 0 0 Hijos
    - 1 1 Hijo
    - 2 2 Hijos
    - ...
    - 25 25 Hijos
    - No informa 99
    - No aplica
- **'PA3_HNVM':** Hijos(as) nacidos vivos: Mujeres
    - 0 0 Hijas
    - 1 1 Hija
    - 2 2 Hijas
    - ...
    - 25 25 Hijos
    - No informa 99
    - No aplica
- **'PA_HNVS':** Hijos(as) sobrevivientes
    - 1 Cuantos
    - 2 No Sabe
    - 9 No Informa
    - No aplica
- **'PA1_THSV':** Hijos(as) sobrevivientes: Total
    - 0 0 Hijos
    - 1 1 Hijo
    - 2 2 Hijos
    - ...
    - 25 25 Hijos
    - No informa 99
    - No aplica
- **'PA2_HSVH':** Hijos(as) sobrevivientes: Hombres
    - 0 0 Hijos
    - 1 1 Hijo
    - 2 2 Hijos
    - ...
    - 25 25 Hijos
    - No informa 99
    - No aplica
- **'PA3_HSVM':** Hijos(as) sobrevivientes: Mujeres
    - 0 0 Hijas
    - 1 1 Hija
    - 2 2 Hijas
    - ...
    - 25 25 Hijos
    - No informa 99
    - No aplica
- **'PA_HFC':** Hijos(as) viven actualmente fuera de Colombia
    - 1 Cuantos
    - 2 No Sabe
    - 9 No Informa
    - No Aplica
- **'PA1_THFC':** Hijos(as) viven actualmente fuera de Colombia: Total
    - 0 0 Hijos
    - 1 1 Hijo
    - 2 2 Hijos
    - ...
    - 25 25 Hijos
    - No informa 99
    - No aplica
- **'PA2_HFCH':** Hijos(as) viven actualmente fuera de Colombia: Hombres
    - 0 0 Hijos
    - 1 1 Hijo
    - 2 2 Hijos
    - ...
    - 25 25 Hijos
    - No informa 99
    - No aplica
- **'PA3_HFCM':** Hijos(as) viven actualmente fuera de Colombia: Mujeres
    - 0 0 Hijos
    - 1 1 Hijo
    - 2 2 Hijos
    - ...
    - 25 25 Hijos
    - No informa 99
    - No aplica
- **'PA_UHNV':** Nacimiento último hijo(a) nacido(a) vivo(a)
    - 1 Si, mes y año
    - 2 No Sabe
    - 9 No informa
    - No aplica
- **'PA1_MES_UHNV':** Nacimiento último hijo(a) nacido(a) vivo(a): Mes
    - 1 Enero
    - 2 Febrero
    - 3 Marzo
    - 4 Abril
    - 5 Mayo
    - 6 Junio
    - 7 Julio
    - 8 Agosto
    - 9 Septiembre
    - 10 Octubre
    - 11 Noviembre
    - 12 Diciembre
    - 99 No informa
    - No aplica
- **'PA2_ANO_UHNV':** Nacimiento último hijo(a) nacido(a) vivo(a): Año
    - 1900
    - ...
    - 2018
    - No informa
    - No aplica

In [34]:
df_per = pd.read_csv(current_dict_files["PER"])
df_per.head()

,TIPO_REG,U_DPTO,U_MPIO,UA_CLASE,U_EDIFICA,COD_ENCUESTAS,U_VIVIENDA,P_NROHOG,P_NRO_PER,P_SEXO,...,PA1_THSV,PA2_HSVH,PA3_HSVM,PA_HFC,PA1_THFC,PA2_HFCH,PA3_HFCM,PA_UHNV,PA1_MES_UHNV,PA2_ANO_UHNV
0,5,5,1,1,6,3626,4,1.0,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5,5,1,1,6,3626,4,1.0,2,2,...,2.0,1.0,1.0,2.0,NaN,NaN,NaN,1.0,3.0,1994.0
2,5,5,1,1,6,3626,4,1.0,3,2,...,1.0,0.0,1.0,2.0,NaN,NaN,NaN,1.0,10.0,2013.0
3,5,5,1,1,6,3626,4,1.0,4,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,5,1,1,6,3626,4,1.0,5,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
df_per.dtypes

TIPO_REG              int64
U_DPTO                int64
U_MPIO                int64
UA_CLASE              int64
U_EDIFICA             int64
COD_ENCUESTAS         int64
U_VIVIENDA            int64
P_NROHOG            float64
P_NRO_PER             int64
P_SEXO                int64
P_EDADR               int64
P_PARENTESCOR       float64
PA1_GRP_ETNIC         int64
PA11_COD_ETNIA      float64
PA12_CLAN           float64
PA21_COD_VITSA      float64
PA22_COD_KUMPA      float64
PA_HABLA_LENG       float64
PA1_ENTIENDE        float64
PB_OTRAS_LENG       float64
PB1_QOTRAS_LENG     float64
PA_LUG_NAC            int64
PA_VIVIA_5ANOS      float64
PA_VIVIA_1ANO       float64
P_ENFERMO           float64
P_QUEHIZO_PPAL      float64
PA_LO_ATENDIERON    float64
PA1_CALIDAD_SERV    float64
CONDICION_FISICA    float64
P_ALFABETA          float64
PA_ASISTENCIA       float64
P_NIVEL_ANOSR       float64
P_TRABAJO           float64
P_EST_CIVIL         float64
PA_HNV              float64
PA1_THNV            

In [32]:
df_per.isnull().sum()

TIPO_REG                  0
U_DPTO                    0
U_MPIO                    0
UA_CLASE                  0
U_EDIFICA                 0
COD_ENCUESTAS             0
U_VIVIENDA                0
P_NROHOG              43296
P_NRO_PER                 0
P_SEXO                    0
P_EDADR                   0
P_PARENTESCOR         43296
PA1_GRP_ETNIC             0
PA11_COD_ETNIA      5937160
PA12_CLAN           5974513
PA21_COD_VITSA      5974648
PA22_COD_KUMPA      5974648
PA_HABLA_LENG       5936495
PA1_ENTIENDE        5962492
PB_OTRAS_LENG       5936495
PB1_QOTRAS_LENG     5966832
PA_LUG_NAC                0
PA_VIVIA_5ANOS        43296
PA_VIVIA_1ANO         43296
P_ENFERMO             43296
P_QUEHIZO_PPAL      5367201
PA_LO_ATENDIERON    5493760
PA1_CALIDAD_SERV    5504225
CONDICION_FISICA      43296
P_ALFABETA           345333
PA_ASISTENCIA        387962
P_NIVEL_ANOSR        345333
P_TRABAJO            768382
P_EST_CIVIL          727218
PA_HNV              3249211
PA1_THNV            

In [33]:
df_per.shape

(5974788, 49)

----

### FALLECIDOS
[Volver](#Exploring_CENSO_dataset)

**Columns:**
- **'TIPO_REG':** Tipo de Registro
- **'U_DPTO':** Departamento
- **'U_MPIO':** Municipio
- **'UA_CLASE':** Clase
    - 1_Cabecera Municipal
    - 2_Centro Poblado
    - 3_Rural Disperso
    - 4_Resto Rural (2 y 3 )
- **'COD_ENCUESTAS':** Codigo Encuesta
- **'U_VIVIENDA':** Número de orden de la Vivienda
- **'F_NROHOG':** Número del hogar en la vivienda
    - 0 Nro_Hogar 00
    - 1 Nro_Hogar 01
    - 2 Nro_Hogar 02
    - ...
    - 35 Nro_Hogar 35
- **'FA1_NRO_FALL':** Número persona fallecida
    - 0 00 Personas fallecidas
    - 1 01 Persona fallecida
    - ...
    - 20 20 Personas fallecidas
- **'FA2_SEXO_FALL':** Sexo Fallecido
    - 1 Hombre
    - 2 Mujer
    - 9 No informa
- **'FA3_EDAD_FALL':** Edad al morir
    - 0
    - 1
    - 2
    - 3
    - 4
    - ...
    - 121
    - 999 No informa
- **'FA4_CERT_DEFUN':** Certificado médico de defunción
    - 1 Si
    - 2 No
    - 3 No sabe
    - 9 No informa

In [28]:
df_fall = pd.read_csv(current_dict_files["FALL"]) #MGN

In [33]:
df_fall.columns.tolist()

['TIPO_REG',
 'U_DPTO',
 'U_MPIO',
 'UA_CLASE',
 'COD_ENCUESTAS',
 'U_VIVIENDA',
 'F_NROHOG',
 'FA1_NRO_FALL',
 'FA2_SEXO_FALL',
 'FA3_EDAD_FALL',
 'FA4_CERT_DEFUN']

In [29]:
df_fall.dtypes

TIPO_REG          int64
U_DPTO            int64
U_MPIO            int64
UA_CLASE          int64
COD_ENCUESTAS     int64
U_VIVIENDA        int64
F_NROHOG          int64
FA1_NRO_FALL      int64
FA2_SEXO_FALL     int64
FA3_EDAD_FALL     int64
FA4_CERT_DEFUN    int64
dtype: object

In [30]:
df_fall.isnull().sum()

TIPO_REG          0
U_DPTO            0
U_MPIO            0
UA_CLASE          0
COD_ENCUESTAS     0
U_VIVIENDA        0
F_NROHOG          0
FA1_NRO_FALL      0
FA2_SEXO_FALL     0
FA3_EDAD_FALL     0
FA4_CERT_DEFUN    0
dtype: int64

In [31]:
df_fall.shape

(31617, 11)

----

### GEOREFERENCIACION
[Volver](#Exploring_CENSO_dataset)

**Columns:**
- **'U_DPTO':** Departamento
- **'U_MPIO':** Municipio
- **'UA_CLASE':** Clase
    - 1_Cabecera Municipal
    - 2_Centro Poblado
    - 3_Rural Disperso
    - 4_Resto Rural (2 y 3 )
- **'UA1_LOCALIDAD':**  Localidad/Comuna
- **'U_SECT_RUR':** Sector Rural
- **'U_SECC_RUR':** Seccion rural
- **'UA2_CPOB':** Centro poblado
- **'U_SECT_URB':** Sector Urbano
- **'U_SECC_URB':** Sección Urbana
- **'U_MZA':** Manzana
- **'U_EDIFICA':** Número orden edificación
- **'COD_ENCUESTAS':** Codigo encuesta
- **'U_VIVIENDA':** Número de orden de la vivienda

In [39]:
df_mgn = pd.read_csv(current_dict_files["MGN"])
df_mgn.head()

,U_DPTO,U_MPIO,UA_CLASE,UA1_LOCALIDAD,U_SECT_RUR,U_SECC_RUR,UA2_CPOB,U_SECT_URB,U_SECC_URB,U_MZA,U_EDIFICA,COD_ENCUESTAS,U_VIVIENDA
0,5,1,1,0,0,0,0,0,0,0,1,950000114,58
1,5,1,1,0,0,0,0,0,0,0,1,950000152,14
2,5,1,1,0,0,0,0,0,0,0,1,950000201,18
3,5,1,1,0,0,0,0,0,0,0,1,950000643,21
4,5,1,1,0,0,0,0,0,0,0,1,950000656,16


In [36]:
df_mgn.dtypes

U_DPTO           int64
U_MPIO           int64
UA_CLASE         int64
UA1_LOCALIDAD    int64
U_SECT_RUR       int64
U_SECC_RUR       int64
UA2_CPOB         int64
U_SECT_URB       int64
U_SECC_URB       int64
U_MZA            int64
U_EDIFICA        int64
COD_ENCUESTAS    int64
U_VIVIENDA       int64
dtype: object

In [37]:
df_mgn.isnull().sum()

U_DPTO           0
U_MPIO           0
UA_CLASE         0
UA1_LOCALIDAD    0
U_SECT_RUR       0
U_SECC_RUR       0
UA2_CPOB         0
U_SECT_URB       0
U_SECC_URB       0
U_MZA            0
U_EDIFICA        0
COD_ENCUESTAS    0
U_VIVIENDA       0
dtype: int64

In [38]:
df_mgn.shape

(2324080, 13)

----

## Divipola
[Volver](#Exploring_CENSO_dataset)

Remove last 5 rows (Are about source and somethigns like that)

In [48]:
import pandas as pd
from sodapy import Socrata

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("www.datos.gov.co", None)

# Example authenticated client (needed for non-public datasets):
# client = Socrata(www.datos.gov.co,
#                  MyAppToken,
#                  userame="user@example.com",
#                  password="AFakePassword")

# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("gdxc-w37w", limit=100000)

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)
results_df.head()

,cod_depto,cod_mpio,dpto,nom_mpio,tipo_municipio
0,05,05001,ANTIOQUIA,MEDELLÍN,Municipio
1,05,05002,ANTIOQUIA,ABEJORRAL,Municipio
2,05,05004,ANTIOQUIA,ABRIAQUÍ,Municipio
3,05,05021,ANTIOQUIA,ALEJANDRÍA,Municipio
4,05,05030,ANTIOQUIA,AMAGÁ,Municipio


In [49]:
results_df = results_df[:-5]
results_df["cod_mpio"] = results_df["cod_mpio"].str[3:]

In [50]:
results_df

,cod_depto,cod_mpio,dpto,nom_mpio,tipo_municipio
0,05,01,ANTIOQUIA,MEDELLÍN,Municipio
1,05,02,ANTIOQUIA,ABEJORRAL,Municipio
2,05,04,ANTIOQUIA,ABRIAQUÍ,Municipio
3,05,21,ANTIOQUIA,ALEJANDRÍA,Municipio
4,05,30,ANTIOQUIA,AMAGÁ,Municipio
...,...,...,...,...,...
1116,97,89,VAUPÉS,YAVARATÉ,Área no municipalizada
1117,99,01,VICHADA,PUERTO CARREÑO,Municipio
1118,99,24,VICHADA,LA PRIMAVERA,Municipio
1119,99,24,VICHADA,SANTA ROSALÍA,Municipio


In [52]:
results_df = results_df.astype({'cod_depto': np.int64,
                                'cod_mpio': np.int64,
                                'dpto': np.dtype('O'),
                                'nom_mpio': np.dtype('O'),
                                'tipo_municipio': np.dtype('O')})

In [54]:
results_df

,cod_depto,cod_mpio,dpto,nom_mpio,tipo_municipio
0,5,1,ANTIOQUIA,MEDELLÍN,Municipio
1,5,2,ANTIOQUIA,ABEJORRAL,Municipio
2,5,4,ANTIOQUIA,ABRIAQUÍ,Municipio
3,5,21,ANTIOQUIA,ALEJANDRÍA,Municipio
4,5,30,ANTIOQUIA,AMAGÁ,Municipio
...,...,...,...,...,...
1116,97,89,VAUPÉS,YAVARATÉ,Área no municipalizada
1117,99,1,VICHADA,PUERTO CARREÑO,Municipio
1118,99,24,VICHADA,LA PRIMAVERA,Municipio
1119,99,24,VICHADA,SANTA ROSALÍA,Municipio


NameError: name 'results_df' is not defined